# LAB 3
**Powered by:** Hapii Denys IP-05

**Topic:** Knowledge-based agents

### Include libraries

Used libs:
- [*XPlot.Plotly*](https://github.com/networkx/networkx)
- Other default | embedded libs:
    - System;
    - System.Math;
    - System.Collections.Generic;

In [181]:
using XPlot.Plotly; 

### Main parameters declaration

Recomendations:

- `nodesCount` should be greater than **1**.
- Square root of `nodesCount` should be an integer.
- `deleteCount` should be less than *Possible Number*
- *`Possible Number`* calculates by formula: `((sqrt(nodesCount)-1)*sqrt(nodesCount)*2)-(nodesCount-1)`
    - example: (5-1) * 5 * 2 - (25 - 1) == 16"
- `agentStart` || `agentEnd` coordinates should be in range: `0 <= *coord* < size`

In [182]:
var nodesCount = 25;
var deleteCount = 10;

var agentStartX = 0;
var agentStartY = 0;

var agentEndX = 4;
var agentEndY = 4;

### Extra functions

In [183]:
static int iToX(int id, int size) { return id % size; }
static int iToY(int id, int size) { return (int)Math.Floor((float)id / size); }
static int XYToI(int x, int y, int size) { return (x % size) + (y * size); }

### Graph Class

In [184]:
public class RoadMap 
{
    public int nodesCount;
    public int deleteCount;
    public int size;
    public List<Graph.Scatter> graph;
    public List<int>[] nodesList;
    int[] nodesArrX;
    int[] nodesArrY;
    Random random;
    
    void InitGraph()
    {
        int id = 0;
        for (int y = 0; y < size; y++)
        {
            for (int x = 0; x < size; x++)
            {
                nodesList[id] = new List<int>();
                if (x != size - 1) nodesList[id].Add(id + 1);
                if (y != size - 1) nodesList[id].Add(id + size);
                if (x != 0) nodesList[id].Add(id - 1);
                if (y != 0) nodesList[id].Add(id - size);
                nodesArrX[id] = x;
                nodesArrY[id] = y;
                id++;
            }
        }
        
        // add markers to graph
        var nodes = new Graph.Scatter()
        {
            x = nodesArrX,
            y = nodesArrY,
            mode = "markers",
            marker = new Graph.Marker() {color = "blue"},
        };
        graph.Add(nodes);
    }
    
    void DFS(int source, List<int>[] adjList, bool[] visited)
    {
        visited[source] = true;
        for (int i = 0; i < adjList[source].Count ; i++) {
            int neighbor = adjList[source][i];
            if (visited[neighbor] == false)
                DFS(neighbor, adjList, visited);
        }
    }

    // check if graph still connected (agent could reach any node)
    bool IsConnected() 
    {
        List<int>[] adjList = nodesList;
        bool[] visited = new bool[nodesCount];
        DFS(0, adjList, visited);
        int count = 0;
        for (int i = 0; i < visited.Length; i++) 
            if (visited[i])
                count++;

        return (nodesCount == count) ? true : false;
    }
    
    void RemoveRandomEdges()
    {
        random = new Random();
        while (deleteCount > 0)
        {
            int num = random.Next(0, nodesList.Length);
            int neighborNode = -1;
            int neighborX = random.Next(-1, 1);
            if (neighborX != 0) neighborNode = num + neighborX;
            else
            {    
                int neighborY = random.Next(-1, 1);
                if (neighborY == 1) neighborY = size;
                if (neighborY == -1) neighborY = -size;
                neighborNode = num - neighborY;
            }

            if (neighborNode < 0 || neighborNode > nodesCount) continue;
        
            if (nodesList[num].Contains(neighborNode))
            {
                nodesList[num].Remove(neighborNode);
                nodesList[neighborNode].Remove(num);
                if (IsConnected()) deleteCount--;
                else 
                {
                    nodesList[num].Add(neighborNode);
                    nodesList[neighborNode].Add(num);
                }
            }
    
        }
    }
    
    void FillGraph(bool debug)
    {
        var counterX = 0;
        var counterY = 0;
        foreach(List<int> node in nodesList)
        {
            if (counterX == size)
            {
                counterX = 0;
                counterY++;
            }
            if (debug) Console.Write("Node " + counterX + "-" + counterY + ": ");
            foreach (int edge in node)
            {
                int edgeX = iToX(edge, size);
                int edgeY = iToY(edge, size);
                if (debug) Console.Write(edgeX + "-" + edgeY + ", ");
                graph.Add(new Graph.Scatter()
                {
                    x = new int[] {counterX, edgeX},
                    y = new int[] {counterY, edgeY},
                    mode = "lines",
                    line = new Graph.Line() {color = "blue"},
                });
            }
            if (debug) Console.WriteLine();
            counterX++;
        }
    }
    
    public RoadMap(int nodesCount, int deleteCount)
    {
        this.nodesCount = nodesCount;
        this.deleteCount = deleteCount;
        this.size = (int)Math.Sqrt(nodesCount);
        nodesList = new List<int>[nodesCount];
        nodesArrX = new int[nodesCount];
        nodesArrY = new int[nodesCount];
        graph = new List<Graph.Scatter>();
        InitGraph();
        RemoveRandomEdges();
        FillGraph(true); // false to turn off printing nodes connections
    }
}

### Create  graph function

In [185]:
PlotlyChart GraphInChart(List<Graph.Scatter> graph) {
    PlotlyChart chart = Chart.Plot(graph);
    chart.WithLegend(false);
    chart.Width = 500;
    chart.Height = 500;
    return chart;
}

### Create and display graph with declared params

In [186]:
var roadMap = new RoadMap(nodesCount, deleteCount);
var chart = GraphInChart(roadMap.graph);
display(chart);

Node 0-0: 1-0, 0-1, 
Node 1-0: 2-0, 0-0, 
Node 2-0: 2-1, 1-0, 
Node 3-0: 4-0, 3-1, 
Node 4-0: 4-1, 3-0, 
Node 0-1: 1-1, 0-2, 0-0, 
Node 1-1: 1-2, 0-1, 
Node 2-1: 3-1, 2-2, 2-0, 
Node 3-1: 3-2, 2-1, 3-0, 
Node 4-1: 4-2, 4-0, 
Node 0-2: 1-2, 0-3, 0-1, 
Node 1-2: 2-2, 1-3, 0-2, 1-1, 
Node 2-2: 3-2, 1-2, 2-1, 
Node 3-2: 4-2, 3-3, 2-2, 3-1, 
Node 4-2: 3-2, 4-1, 4-3, 
Node 0-3: 0-2, 0-4, 
Node 1-3: 2-3, 1-2, 
Node 2-3: 3-3, 2-4, 1-3, 
Node 3-3: 3-4, 2-3, 3-2, 
Node 4-3: 4-2, 
Node 0-4: 0-3, 
Node 1-4: 2-4, 
Node 2-4: 3-4, 1-4, 2-3, 
Node 3-4: 2-4, 3-3, 4-4, 
Node 4-4: 3-4, 


### Agent Abstraction

In [187]:
public class Agent 
{
    int currentX;
    int currentY;
    int currentI;
    int endX;
    int endY;
    int endI;
    int prevX;
    int prevY;
    Stack<int> roadsMemory;
    List<int> visited;
    RoadMap roadMap;
    public List<Graph.Scatter> graph;
    bool debugMode;
    int nextNode;
    
    void DebugLocation()
    {
        if (debugMode) Console.Write("> (" + currentX + "," + currentY + ") ");
    }
    
    public Agent(RoadMap map, int startX, int startY, int endX, int endY, bool debugMode)
    {
        roadMap = map;
        currentX = startX;
        currentY = startY;
        currentI = XYToI(currentX, currentY, roadMap.size); // transform 2d into 1d
        this.endX = endX;
        this.endY = endY;
        endI = XYToI(endX, endY, roadMap.size);// transform 2d into 1d
        prevX = currentX;
        prevY = currentY;
        roadsMemory = new Stack<int>(); // stack with remembering `correct` path of agent
        visited = new List<int>(); // list of all visited nodes
        graph = map.graph; // make copy for new graph
        this.debugMode = debugMode;
        // put agent at start point:
        DebugLocation();
        roadsMemory.Push(currentI);
        visited.Add(currentI);
    }
    
    void Rotate()
    {
        roadsMemory.Pop();
    }
    
    void ChooseDirection()
    {
        roadsMemory.Push(nextNode); // add node to path
        visited.Add(nextNode); // mark as visited
    }
    
    void Move()
    {
        prevX = currentX;
        prevY = currentY;
        currentI = roadsMemory.Peek();
        currentX = iToX(currentI, roadMap.size);
        currentY = iToY(currentI, roadMap.size);
        graph.Add(new Graph.Scatter()
        {
            x = new int[] {prevX, currentX},
            y = new int[] {prevY, currentY},
            mode = "lines",
            line = new Graph.Line() {color = "yellow"},
        });  
    }
    
    bool CheckImpasseSign(int node)
    {
        return (roadMap.nodesList[node].Count == 1) ? true : false;
    }
    
    void Tell()
    {
        nextNode = -1;
        int heuristicCurr = Math.Abs(currentX - endX) + 
            Math.Abs(currentY - endY);
        foreach(int nextI in roadMap.nodesList[currentI]) // check existing roads from current node
        {
            if(!visited.Contains(nextI))
            {
                if (nextI == endI) 
                {
                    nextNode = nextI;
                    break;
                }
                if (CheckImpasseSign(nextI)) continue;
                if (nextNode == -1) nextNode = nextI;
                int heuristicNext = Math.Abs(iToX(nextI, roadMap.size) - endX) + 
                    Math.Abs(iToY(nextI, roadMap.size) - endY);
                if (heuristicNext < heuristicCurr)
                    nextNode = nextI;
            }
        }
    }
    
    Action Ask()
    {
            if (nextNode == -1)
                return Rotate;
            else
                return ChooseDirection;
    }
    
    public void MoveToGoal()
    {
        while (currentI != endI) 
        {
            Tell();
            Action Act = Ask();
            Act();
            Move();
            DebugLocation();
        }
    }
    
    public void FillGraphWithPath()
    {
        prevX = endX;
        prevY = endY;
        foreach(int node in roadsMemory)
        {
            currentX = iToX(node, roadMap.size);
            currentY = iToY(node, roadMap.size);
            graph.Add(new Graph.Scatter()
            {
                x = new int[] {prevX, currentX},
                y = new int[] {prevY, currentY},
                mode = "lines",
                line = new Graph.Line() {color = "red"},
            }); 
            prevX = currentX;
            prevY = currentY;
        }
    }
}

### Create and display graph with path

In [188]:
var agent = new Agent(roadMap, agentStartX, agentStartY, agentEndX, agentEndY, true);
agent.MoveToGoal();
agent.FillGraphWithPath();
var chart = GraphInChart(agent.graph);
display(chart);

> (0,0) > (0,1) > (0,2) > (0,3) > (0,2) > (1,2) > (1,3) > (2,3) > (2,4) > (3,4) > (4,4) 